In [1]:
from pathlib import Path
import shutil

In [2]:
# 1, load the data:
data_dirs = Path('../../data/FBL/train').glob('*IM.tiff')
dst_dir = Path('../../data/FBL/train_im')
dst_dir.mkdir(exist_ok=True)
for data_dir in data_dirs:
    shutil.copy(data_dir,dst_dir)

In [41]:
# 2, calculate the ssim:
prd_path = Path('/mnt/eternus/users/Yu/project/data_compression/CompressAI/hello.tiff')
input_path = Path('/mnt/eternus/users/Yu/project/data_compression/data/labelfree_2d/test/7636_0_IM.tiff')

In [8]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path
import os
import sys
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import monai
import torch

def normalizeItensity(image):
    # Convert the image data to a floating-point data type
    img = image.astype(np.float32)
    # Convert the image to a toch Tensor
    img = torch.from_numpy(img)
    # Normalize the intensity of the image using the MONAI NormalizeIntensity transform
    normalize_intensity = monai.transforms.NormalizeIntensity()
    img_normalized = normalize_intensity(img)
    # Convert the normalized image back to a numpy array
    return img_normalized.numpy()

def transform_img(image):
    img = image.astype(np.float32) / 65535
    print(type(img))
    return img

def compare_images(path1, path2, gt=False):
    # Load the two images
    image1 = AICSImage(path1).get_image_data('YX') 
    image2 = AICSImage(path2).get_image_data('YX') 
    if gt:
        # image1 = image1.astype(np.float32)
        # # Scale the values in image1 to the range [0, 65535]
        # scaled_image = (image1 / np.max(image1) * 65535).round().astype(np.uint16)
        # image1 = scaled_image
        image1 = transform_img(image1)
        image2 = transform_img(image2)
    # Calculate metrics
    mse = np.sum((image1 - image2) ** 2)/(924*624)
    ssim_value = ssim(image1, image2)
    psnr_value = psnr(image1, image2)
    corr = np.corrcoef(image1.ravel(), image2.ravel())[0, 1]
    # psnr = 10 * np.log10(1 / (mse + 0.000001))
    return mse, ssim_value, psnr_value, corr

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
from compressai.zoo.image import cfgs
from compressai.zoo import image_models

holdout_path_2d = Path("/mnt/eternus/users/Yu/project/data_compression/data/labelfree_2d/test/")
for image in holdout_path_2d.glob('*IM*.tiff'): 
    model_name= 'bmshj2018-factorized'+'_'+'mse'+'_'+'8'
    path_encoded= image.parent/model_name/str(image.stem+'_encoded')
    path_encoded.parent.mkdir(exist_ok=True, parents = True)
    path_decoded= image.parent/model_name/str(image.stem+'_decoded.tiff')
    path_decoded.parent.mkdir(exist_ok=True, parents = True)
    if not path_decoded.is_file():
        !python3 codec.py encode {image} -o {path_encoded} --model bmshj2018-factorized -q 8 -m mse --cuda
        !python3 codec.py decode {path_encoded} -o {path_decoded} --cuda

In [10]:
decoded_paths = sorted(Path('/mnt/eternus/users/Yu/project/data_compression/data/labelfree_2d/test/bmshj2018-factorized_mse_8/').glob('*.tiff'))
holdout_paths = sorted(Path("/mnt/eternus/users/Yu/project/data_compression/data/labelfree_2d/test/").glob('*.tiff'))

In [11]:
mse_value = AverageMeter()
ssim_value = AverageMeter()
psnr_value = AverageMeter()
corr_value = AverageMeter()
for i, (decode_path, holdout_path) in enumerate(zip(decoded_paths, holdout_paths)):
    tmp_mse, tmp_ssim, tmp_psnr, tmp_corr = compare_images(decode_path, holdout_path)
    mse_value.update(tmp_mse)
    ssim_value.update(tmp_ssim)
    psnr_value.update(tmp_psnr)
    corr_value.update(tmp_corr)
print(mse_value.avg, ssim_value.avg, psnr_value.avg, corr_value.avg)

Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
Setting data_range based on im1.dtype. data_range = 65535. Please specify data_r

25140.912787854504 0.9823943768024408 45.09721279002597 0.9912424183985047


Setting data_range based on im1.dtype. data_range = 65535. Please specify data_range explicitly to avoid mistakes.
